In [1]:
! pip3 install redis
! pip3 install pymongo
! pip3 install confluent_kafka

    100% |################################| 71kB 990kB/s ta 0:00:01
    100% |################################| 450kB 1.4MB/s ta 0:00:011
    100% |################################| 7.4MB 218kB/s ta 0:00:011


In [2]:
from confluent_kafka import Consumer, KafkaException
import redis
import pymongo
import confluent_kafka
import logging
import sys
import json

In [3]:
def connect_redis():
    
    TICKER  = 4
    
    return redis.StrictRedis(

        host='127.0.0.1',

        port=6379,
    
        db=TICKER)

In [4]:
redis_conn = connect_redis()

In [19]:
def connect_mongo():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/", 
                                   username='root',
                                   password='password')

    mydb = myclient["upbit"]
    mycol = mydb["ticker"]
    
    return mycol

In [20]:
mongo_col = connect_mongo()

In [21]:
def connect_kafka_as_a_consumer():
    # Create logger for consumer (logs will be emitted when poll() is called)
    logger = logging.getLogger('consumer')
    logger.setLevel(logging.DEBUG)
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter('%(asctime)-15s %(levelname)-8s %(message)s'))
    logger.addHandler(handler)
    
    conf = {'bootstrap.servers': "localhost:9092,localhost:9093,localhost:9094",
        'group.id': "ticker_saver",
        'auto.offset.reset': 'earliest'}
    
    return Consumer(conf, logger=logger)

In [22]:
consumer = connect_kafka_as_a_consumer()

In [23]:
def print_assignment(consumer, partitions):
        print('Assignment:', partitions)

In [24]:
def data_filtering(data):
    del data["trade_date"]
    del data["trade_time"]
    del data["trade_status"]
    del data["market_state_for_ios"]
    del data["delisting_date"]
    del data["market_warning"]
    del data["stream_type"]
    return data

In [25]:
def extract_key_of_data(data):
    return data["timestamp"]

In [26]:
def save_redis(conn,key,data):
    data = json.dumps(data)
    print ('Set Record:', conn.set(key, data))

In [28]:
def save_mongo(col,data):
    x = col.insert_one(data)
    print(x.inserted_id)

In [29]:
consumer.subscribe(["ticker"], on_assign=print_assignment)

In [30]:
count = 0

while True:
    try:
        msg = consumer.poll(1.0)
            
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            # Proper message
            data = json.loads(msg.value()) 
            data = data_filtering(data)
            key = extract_key_of_data(data)
            save_redis(redis_conn,key,data)
            save_mongo(mongo_col,data)
        if count > 10000:
            consumer.close()
            break
        count += 1
    except KeyboardInterrupt:
        sys.stderr.write('%% Aborted by user\n')
        consumer.close()
        break

Assignment: [TopicPartition{topic=ticker,partition=0,offset=-1001,error=None}]
Set Record: True
5de952b8a82019c718212c8b
Set Record: True
5de952b8a82019c718212c8c
Set Record: True
5de952b8a82019c718212c8d
Set Record: True
5de952b8a82019c718212c8e
Set Record: True
5de952b8a82019c718212c8f
Set Record: True
5de952b8a82019c718212c90
Set Record: True
5de952b8a82019c718212c91
Set Record: True
5de952b8a82019c718212c92
Set Record: True
5de952b8a82019c718212c93
Set Record: True
5de952b8a82019c718212c94
Set Record: True
5de952b8a82019c718212c95
Set Record: True
5de952b8a82019c718212c96
Set Record: True
5de952b8a82019c718212c97
Set Record: True
5de952b8a82019c718212c98
Set Record: True
5de952b8a82019c718212c99
Set Record: True
5de952b8a82019c718212c9a
Set Record: True
5de952b8a82019c718212c9b
Set Record: True
5de952b8a82019c718212c9c
Set Record: True
5de952b8a82019c718212c9d
Set Record: True
5de952b8a82019c718212c9e
Set Record: True
5de952b8a82019c718212c9f
Set Record: True
5de952b8a82019c718212c

Set Record: True
5de952b8a82019c718212d8b
Set Record: True
5de952b8a82019c718212d8c
Set Record: True
5de952b8a82019c718212d8d
Set Record: True
5de952b8a82019c718212d8e
Set Record: True
5de952b8a82019c718212d8f
Set Record: True
5de952b8a82019c718212d90
Set Record: True
5de952b8a82019c718212d91
Set Record: True
5de952b8a82019c718212d92
Set Record: True
5de952b8a82019c718212d93
Set Record: True
5de952b8a82019c718212d94
Set Record: True
5de952b8a82019c718212d95
Set Record: True
5de952b8a82019c718212d96
Set Record: True
5de952b8a82019c718212d97
Set Record: True
5de952b8a82019c718212d98
Set Record: True
5de952b8a82019c718212d99
Set Record: True
5de952b8a82019c718212d9a
Set Record: True
5de952b8a82019c718212d9b
Set Record: True
5de952b8a82019c718212d9c
Set Record: True
5de952b8a82019c718212d9d
Set Record: True
5de952b8a82019c718212d9e
Set Record: True
5de952b8a82019c718212d9f
Set Record: True
5de952b8a82019c718212da0
Set Record: True
5de952b8a82019c718212da1
Set Record: True
5de952b8a82019c71

Set Record: True
5de952b8a82019c718212e9d
Set Record: True
5de952b8a82019c718212e9e
Set Record: True
5de952b8a82019c718212e9f
Set Record: True
5de952b8a82019c718212ea0
Set Record: True
5de952b8a82019c718212ea1
Set Record: True
5de952b8a82019c718212ea2
Set Record: True
5de952b8a82019c718212ea3
Set Record: True
5de952b8a82019c718212ea4
Set Record: True
5de952b8a82019c718212ea5
Set Record: True
5de952b8a82019c718212ea6
Set Record: True
5de952b8a82019c718212ea7
Set Record: True
5de952b8a82019c718212ea8
Set Record: True
5de952b8a82019c718212ea9
Set Record: True
5de952b8a82019c718212eaa
Set Record: True
5de952b8a82019c718212eab
Set Record: True
5de952b8a82019c718212eac
Set Record: True
5de952b8a82019c718212ead
Set Record: True
5de952b8a82019c718212eae
Set Record: True
5de952b8a82019c718212eaf
Set Record: True
5de952b8a82019c718212eb0
Set Record: True
5de952b8a82019c718212eb1
Set Record: True
5de952b8a82019c718212eb2
Set Record: True
5de952b8a82019c718212eb3
Set Record: True
5de952b8a82019c71

Set Record: True
5de952b8a82019c718212f9d
Set Record: True
5de952b8a82019c718212f9e
Set Record: True
5de952b8a82019c718212f9f
Set Record: True
5de952b8a82019c718212fa0
Set Record: True
5de952b8a82019c718212fa1
Set Record: True
5de952b8a82019c718212fa2
Set Record: True
5de952b8a82019c718212fa3
Set Record: True
5de952b8a82019c718212fa4
Set Record: True
5de952b8a82019c718212fa5
Set Record: True
5de952b8a82019c718212fa6
Set Record: True
5de952b8a82019c718212fa7
Set Record: True
5de952b8a82019c718212fa8
Set Record: True
5de952b8a82019c718212fa9
Set Record: True
5de952b8a82019c718212faa
Set Record: True
5de952b8a82019c718212fab
Set Record: True
5de952b8a82019c718212fac
Set Record: True
5de952b8a82019c718212fad
Set Record: True
5de952b8a82019c718212fae
Set Record: True
5de952b8a82019c718212faf
Set Record: True
5de952b8a82019c718212fb0
Set Record: True
5de952b8a82019c718212fb1
Set Record: True
5de952b8a82019c718212fb2
Set Record: True
5de952b8a82019c718212fb3
Set Record: True
5de952b8a82019c71

5de952b8a82019c7182130b0
Set Record: True
5de952b8a82019c7182130b1
Set Record: True
5de952b8a82019c7182130b2
Set Record: True
5de952b8a82019c7182130b3
Set Record: True
5de952b8a82019c7182130b4
Set Record: True
5de952b8a82019c7182130b5
Set Record: True
5de952b8a82019c7182130b6
Set Record: True
5de952b8a82019c7182130b7
Set Record: True
5de952b8a82019c7182130b8
Set Record: True
5de952b8a82019c7182130b9
Set Record: True
5de952b8a82019c7182130ba
Set Record: True
5de952b8a82019c7182130bb
Set Record: True
5de952b8a82019c7182130bc
Set Record: True
5de952b8a82019c7182130bd
Set Record: True
5de952b8a82019c7182130be
Set Record: True
5de952b8a82019c7182130bf
Set Record: True
5de952b8a82019c7182130c0
Set Record: True
5de952b8a82019c7182130c1
Set Record: True
5de952b8a82019c7182130c2
Set Record: True
5de952b8a82019c7182130c3
Set Record: True
5de952b8a82019c7182130c4
Set Record: True
5de952b8a82019c7182130c5
Set Record: True
5de952b8a82019c7182130c6
Set Record: True
5de952b8a82019c7182130c7
Set Recor

...

Set Record: True
5de96b5aa82019c718214ac9
Set Record: True
5de96b5aa82019c718214aca
Set Record: True
5de96b5aa82019c718214acb
Set Record: True
5de96b5aa82019c718214acc
Set Record: True
5de96b5ba82019c718214acd
Set Record: True
5de96b5da82019c718214ace
Set Record: True
5de96b5da82019c718214acf
Set Record: True
5de96b5ea82019c718214ad0
Set Record: True
5de96b5fa82019c718214ad1
Set Record: True
5de96b60a82019c718214ad2
Set Record: True
5de96b60a82019c718214ad3
Set Record: True
5de96b62a82019c718214ad4
Set Record: True
5de96b62a82019c718214ad5
Set Record: True
5de96b64a82019c718214ad6
Set Record: True
5de96b64a82019c718214ad7
Set Record: True
5de96b65a82019c718214ad8
Set Record: True
5de96b66a82019c718214ad9
Set Record: True
5de96b67a82019c718214ada
Set Record: True
5de96b68a82019c718214adb
Set Record: True
5de96b69a82019c718214adc
Set Record: True
5de96b69a82019c718214add
Set Record: True
5de96b6ba82019c718214ade
Set Record: True
5de96b6ba82019c718214adf
Set Record: True
5de96b6ca82019c71

Set Record: True
5de96c3fa82019c718214b8d
Set Record: True
5de96c3fa82019c718214b8e
Set Record: True
5de96c40a82019c718214b8f
Set Record: True
5de96c47a82019c718214b90
Set Record: True
5de96c53a82019c718214b91
Set Record: True
5de96c5ea82019c718214b92
Set Record: True
5de96c60a82019c718214b93
Set Record: True
5de96c60a82019c718214b94
Set Record: True
5de96c64a82019c718214b95
Set Record: True
5de96c68a82019c718214b96
Set Record: True
5de96c71a82019c718214b97
Set Record: True
5de96c71a82019c718214b98
Set Record: True
5de96c73a82019c718214b99
Set Record: True
5de96c73a82019c718214b9a
Set Record: True
5de96c7aa82019c718214b9b
Set Record: True
5de96c7ca82019c718214b9c
Set Record: True
5de96c7da82019c718214b9d
Set Record: True
5de96c7fa82019c718214b9e
Set Record: True
5de96c81a82019c718214b9f
Set Record: True
5de96c82a82019c718214ba0
Set Record: True
5de96c86a82019c718214ba1
Set Record: True
5de96c87a82019c718214ba2
Set Record: True
5de96c87a82019c718214ba3
Set Record: True
5de96c87a82019c71

Set Record: True
5de96e12a82019c718214c51
Set Record: True
5de96e1ea82019c718214c52
Set Record: True
5de96e20a82019c718214c53
Set Record: True
5de96e24a82019c718214c54
Set Record: True
5de96e27a82019c718214c55
Set Record: True
5de96e3ea82019c718214c56
Set Record: True
5de96e47a82019c718214c57
Set Record: True
5de96e4aa82019c718214c58
Set Record: True
5de96e4ba82019c718214c59
Set Record: True
5de96e50a82019c718214c5a
Set Record: True
5de96e55a82019c718214c5b
Set Record: True
5de96e56a82019c718214c5c
Set Record: True
5de96e5ca82019c718214c5d
Set Record: True
5de96e62a82019c718214c5e
Set Record: True
5de96e62a82019c718214c5f
Set Record: True
5de96e62a82019c718214c60
Set Record: True
5de96e62a82019c718214c61
Set Record: True
5de96e62a82019c718214c62
Set Record: True
5de96e6ba82019c718214c63
Set Record: True
5de96e6ca82019c718214c64
Set Record: True
5de96e6ea82019c718214c65
Set Record: True
5de96e6ea82019c718214c66
Set Record: True
5de96e6fa82019c718214c67
Set Record: True
5de96e6fa82019c71

Set Record: True
5de97082a82019c718214d15
Set Record: True
5de97083a82019c718214d16
Set Record: True
5de97083a82019c718214d17
Set Record: True
5de97083a82019c718214d18
Set Record: True
5de97087a82019c718214d19
Set Record: True
5de97087a82019c718214d1a
Set Record: True
5de97087a82019c718214d1b
Set Record: True
5de97087a82019c718214d1c
Set Record: True
5de97087a82019c718214d1d
Set Record: True
5de97089a82019c718214d1e
Set Record: True
5de9708da82019c718214d1f
Set Record: True
5de9708da82019c718214d20
Set Record: True
5de97092a82019c718214d21
Set Record: True
5de97095a82019c718214d22
Set Record: True
5de97096a82019c718214d23
Set Record: True
5de9709ba82019c718214d24
Set Record: True
5de970a3a82019c718214d25
Set Record: True
5de970b4a82019c718214d26
Set Record: True
5de970d2a82019c718214d27
Set Record: True
5de970e2a82019c718214d28
Set Record: True
5de970e2a82019c718214d29
Set Record: True
5de970e2a82019c718214d2a
Set Record: True
5de970e2a82019c718214d2b
Set Record: True
5de970f0a82019c71

%% Aborted by user
